Hi,

This is a draft code that loads the pretrained model and predicts a given input. 
The notebook should run in less than five minutes.

Keep in mind that you should download the following folder (my saved model) and put it in the same folder as your code.

https://mega.nz/folder/MmB1gIIT#8ilUTK1-BO80aoXxKOIhpg

-Moradnejad

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
!pip install transformers
!pip install sentencepiece
!pip install tensorflow-gpu

# Load the model

In [25]:
import keras

model = keras.models.load_model("/content/drive/Shareddrives/NLP Final Project/Vijay Colab/ColBERT/colbert-trained/")
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_19 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_20 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_21 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 input_22 (InputLayer)          [(None, 20)]         0           []                               
                                                                                       

# complete code

In [26]:
import subprocess
from ast import literal_eval

def run(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    out, err = process.communicate()
    print(out.decode('utf-8').strip())

print('# CPU')
run('cat /proc/cpuinfo | egrep -m 1 "^model name"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu MHz"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu cores"')

print('# RAM')
run('cat /proc/meminfo | egrep "^MemTotal"')

print('# GPU')
run('lspci | grep VGA')

print('# OS')
run('uname -a')

# CPU
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
cpu MHz		: 2299.998
cpu cores	: 1
# RAM
MemTotal:       13302920 kB
# GPU

# OS
Linux 6d03cba5c680 5.4.144+ #1 SMP Tue Dec 7 09:58:10 PST 2021 x86_64 x86_64 x86_64 GNU/Linux


In [27]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# import tensorflow_hub as hub
import tensorflow as tf
# import bert_tokenization as tokenization
import tensorflow.keras.backend as K
from tensorflow import keras 

import os
from scipy.stats import spearmanr
from math import floor, ceil
# from transformers import *

import seaborn as sns
import string
import re    #for regex

np.set_printoptions(suppress=True)
print(tf.__version__)

2.8.0


# Prep / tokenizer

#### 1. Read data and tokenizer

Read tokenizer and data, as well as defining the maximum sequence length that will be used for the input to Bert (maximum is usually 512 tokens)

In [28]:
training_sample_count = 1000 # 4000
test_count = 1000

MAX_SENTENCE_LENGTH = 20
MAX_SENTENCES = 5
MAX_LENGTH = 100

In [29]:
!ls /content/drive/Shareddrives/NLP\ Final\ Project/Vijay\ Colab/data/english

dataset.csv  dev.csv  Readme.md  test.csv  train.csv


### original dataset

In [41]:
df = pd.read_csv('/content/drive/Shareddrives/NLP Final Project/Vijay Colab/data/english/dataset.csv')

df_train = pd.read_csv('/content/drive/Shareddrives/NLP Final Project/Vijay Colab/data/english/train.csv')
display(df_train.head(3))
df_train = df_train[:training_sample_count]

df_test = pd.read_csv('/content/drive/Shared drives/NLP Final Project/Dataset/English_balanced_test_task1.csv')
display(df_test.head(3))
# df_test = df_test[:test_count] # ORIGINAL CODE


,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
1,Watch: darvish gave hitter whiplash with slow ...,False
2,What do you call a turtle without its shell? d...,True


,Unnamed: 0,id,joke,label
0,0,13585,New regulations mean new jobs for new regulators.,0
1,1,12656,I have enough money for the rest of my life......,1
2,2,2957,"Yesterday today is tomorrow, tomorrow today is...",0


In [42]:
df_test.drop(columns=['Unnamed: 0', 'id'], inplace=True)
display(df_test.head(3))

,joke,label
0,New regulations mean new jobs for new regulators.,0
1,I have enough money for the rest of my life......,1
2,"Yesterday today is tomorrow, tomorrow today is...",0


In [43]:
df_test.rename(columns ={'joke': 'text', 'label':'humor'}, inplace=True)
display(df_test.head(3))

,text,humor
0,New regulations mean new jobs for new regulators.,0
1,I have enough money for the rest of my life......,1
2,"Yesterday today is tomorrow, tomorrow today is...",0


In [44]:
test_df_y = df_test.copy()
del df_test['humor']

df_sub = test_df_y.copy()

print(len(df),len(df_train),len(df_test))
display(df_train.head())
display(df_test.head())

200000 1000 1600


,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
1,Watch: darvish gave hitter whiplash with slow ...,False
2,What do you call a turtle without its shell? d...,True
3,5 reasons the 2016 election feels so personal,False
4,"Pasco police shot mexican migrant from behind,...",False


,text
0,New regulations mean new jobs for new regulators.
1,I have enough money for the rest of my life......
2,"Yesterday today is tomorrow, tomorrow today is..."
3,waiter! One customer shouts that the ad says y...
4,I think your problem is low self-esteem. Commo...


In [45]:
print(list(df_train.columns))

['text', 'humor']


In [46]:
output_categories = list(df_train.columns[[1]])
input_categories = list(df_train.columns[[0]])

TARGET_COUNT = len(output_categories)

print('\ninput categories:\n\t', input_categories)
print('\noutput TARGET_COUNT:\n\t', TARGET_COUNT)
print('\noutput categories:\n\t', output_categories)


input categories:
	 ['text']

output TARGET_COUNT:
	 1

output categories:
	 ['humor']


## 2. Preprocessing functions

These are some functions that will be used to preprocess the raw text data into useable Bert inputs.<br>


In [47]:
from transformers import BertTokenizer

MODEL_TYPE = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE)

In [48]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [49]:
def return_id(str1, str2, truncation_strategy, length):

    inputs = tokenizer.encode_plus(str1, str2,
        add_special_tokens=True,
        max_length=length,
        truncation_strategy=truncation_strategy)

    input_ids =  inputs["input_ids"]
    input_masks = [1] * len(input_ids)
    input_segments = inputs["token_type_ids"]
    padding_length = length - len(input_ids)
    padding_id = tokenizer.pad_token_id
    input_ids = input_ids + ([padding_id] * padding_length)
    input_masks = input_masks + ([0] * padding_length)
    input_segments = input_segments + ([0] * padding_length)

    return [input_ids, input_masks, input_segments]


def compute_input_arrays(df, columns, tokenizer):
    model_input = []
    for xx in range((MAX_SENTENCES*3)+3):
        model_input.append([])
    
    for _, row in tqdm(df[columns].iterrows()):
        i = 0
        
        # sent
        sentences = sent_tokenize(row.text)
        for xx in range(MAX_SENTENCES):
            s = sentences[xx] if xx<len(sentences) else ''
            ids_q, masks_q, segments_q = return_id(s, None, 'longest_first', MAX_SENTENCE_LENGTH)
            model_input[i].append(ids_q)
            i+=1
            model_input[i].append(masks_q)
            i+=1
            model_input[i].append(segments_q)
            i+=1
        
        # full row
        ids_q, masks_q, segments_q = return_id(row.text, None, 'longest_first', MAX_LENGTH)
        model_input[i].append(ids_q)
        i+=1
        model_input[i].append(masks_q)
        i+=1
        model_input[i].append(segments_q)
        
    for xx in range((MAX_SENTENCES*3)+3):
        model_input[xx] = np.asarray(model_input[xx], dtype=np.int32)
        
    print(model_input[0].shape)
    return model_input



In [50]:
inputs      = compute_input_arrays(df_train, input_categories, tokenizer)
test_inputs = compute_input_arrays(df_test, input_categories, tokenizer)

0it [00:00, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


(1000, 20)


0it [00:00, ?it/s]

(1600, 20)


In [51]:
print(len(inputs), len(inputs[0]), len(inputs[0][0]))

# check out input for 7th row
xx = 7
print(df_train.iloc[xx,0])
print(sent_tokenize(df_train.iloc[xx,0]))
inputs[0][xx], inputs[3][xx], inputs[6][xx], inputs[15][xx]

18 1000 20
Why do native americans hate it when it rains in april? because it brings mayflowers.
['Why do native americans hate it when it rains in april?', 'because it brings mayflowers.']


(array([  101,  2339,  2079,  3128,  4841,  5223,  2009,  2043,  2009,
        15811,  1999,  2258,  1029,   102,     0,     0,     0,     0,
            0,     0], dtype=int32),
 array([  101,  2138,  2009,  7545,  2089, 14156,  2015,  1012,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0], dtype=int32),
 array([101, 102,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0], dtype=int32),
 array([  101,  2339,  2079,  3128,  4841,  5223,  2009,  2043,  2009,
        15811,  1999,  2258,  1029,  2138,  2009,  7545,  2089, 14156,
         2015,  1012,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,

In [52]:
def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

outputs = compute_output_arrays(df_train, output_categories)
outputs[:3]

array([[False],
       [False],
       [ True]])

## 5. Training, validation and testing

Loops over the folds in gkf and trains each fold for 3 epochs --- with a learning rate of 3e-5 and batch_size of 6. A simple binary crossentropy is used as the objective-/loss-function. 

In [53]:
# Evaluation Metrics
import sklearn
def print_evaluation_metrics(y_true, y_pred, label='', is_regression=True, label2=''):
    print('==================', label2)
    ### For regression
    if is_regression:
        print('mean_absolute_error',label,':', sklearn.metrics.mean_absolute_error(y_true, y_pred))
        print('mean_squared_error',label,':', sklearn.metrics.mean_squared_error(y_true, y_pred))
        print('r2 score',label,':', sklearn.metrics.r2_score(y_true, y_pred))
        #     print('max_error',label,':', sklearn.metrics.max_error(y_true, y_pred))
        return sklearn.metrics.mean_squared_error(y_true, y_pred)
    else:
        ### FOR Classification
#         print('balanced_accuracy_score',label,':', sklearn.metrics.balanced_accuracy_score(y_true, y_pred))
#         print('average_precision_score',label,':', sklearn.metrics.average_precision_score(y_true, y_pred))
#         print('balanced_accuracy_score',label,':', sklearn.metrics.balanced_accuracy_score(y_true, y_pred))
#         print('accuracy_score',label,':', sklearn.metrics.accuracy_score(y_true, y_pred))
        print('f1_score',label,':', sklearn.metrics.f1_score(y_true, y_pred))
        
        matrix = sklearn.metrics.confusion_matrix(y_true, y_pred)
        print(matrix)
        TP,TN,FP,FN = matrix[1][1],matrix[0][0],matrix[0][1],matrix[1][0]
        Accuracy = (TP+TN)/(TP+FP+FN+TN)
        Precision = TP/(TP+FP)
        Recall = TP/(TP+FN)
        F1 = 2*(Recall * Precision) / (Recall + Precision)
        print('Acc', Accuracy, 'Prec', Precision, 'Rec', Recall, 'F1',F1)
        return sklearn.metrics.accuracy_score(y_true, y_pred)

print_evaluation_metrics([1,0], [0.9,0.1], '', True)
print_evaluation_metrics([1,0], [1,1], '', False)

mean_absolute_error  : 0.09999999999999999
mean_squared_error  : 0.009999999999999998
r2 score  : 0.96
f1_score  : 0.6666666666666666
[[0 1]
 [0 1]]
Acc 0.5 Prec 0.5 Rec 1.0 F1 0.6666666666666666


0.5

### Loss function selection
Regression problem between 0 and 1, so binary_crossentropy and mean_absolute_error seem good.

Here are the explanations: https://www.dlology.com/blog/how-to-choose-last-layer-activation-and-loss-function/

In [54]:
valid_inputs = inputs
valid_outputs = outputs

In [55]:
preds = model.predict(valid_inputs)


In [56]:
len(valid_inputs[0])

1000

In [57]:
print(valid_outputs.shape, preds.shape)
print_evaluation_metrics(np.array(valid_outputs), np.array(preds), '')

(1000, 1) (1000, 1)
mean_absolute_error  : 0.008370926
mean_squared_error  : 0.005929642
r2 score  : 0.9762767835880616


0.005929642

In [58]:
test_preds = model.predict(test_inputs)

In [59]:
df_sub.head()

,text,humor
0,New regulations mean new jobs for new regulators.,0
1,I have enough money for the rest of my life......,1
2,"Yesterday today is tomorrow, tomorrow today is...",0
3,waiter! One customer shouts that the ad says y...,1
4,I think your problem is low self-esteem. Commo...,0


## Binary submission

In [60]:
for split in np.arange(0.1, 0.99, 0.1).tolist():
    df_sub['pred_bi'] = (test_preds > split)

    print_evaluation_metrics(df_sub['humor'], df_sub['pred_bi'], '', False, 'SPLIT on '+str(split))

    df_sub.to_csv('sub3.csv', index=False)
    df_sub.head()

================== SPLIT on 0.1
f1_score  : 0.6691887347625054
[[ 17 783]
 [  4 796]]
Acc 0.508125 Prec 0.5041165294490184 Rec 0.995 F1 0.6691887347625054
================== SPLIT on 0.2
f1_score  : 0.6686315789473685
[[ 19 781]
 [  6 794]]
Acc 0.508125 Prec 0.5041269841269841 Rec 0.9925 F1 0.6686315789473685
================== SPLIT on 0.30000000000000004
f1_score  : 0.6683522966708808
[[ 20 780]
 [  7 793]]
Acc 0.508125 Prec 0.5041322314049587 Rec 0.99125 F1 0.6683522966708808
================== SPLIT on 0.4
f1_score  : 0.6683544303797467
[[ 22 778]
 [  8 792]]
Acc 0.50875 Prec 0.5044585987261146 Rec 0.99 F1 0.6683544303797467
================== SPLIT on 0.5
f1_score  : 0.6692047377326565
[[ 27 773]
 [  9 791]]
Acc 0.51125 Prec 0.5057544757033248 Rec 0.98875 F1 0.6692047377326565
================== SPLIT on 0.6
f1_score  : 0.6706231454005935
[[ 32 768]
 [  9 791]]
Acc 0.514375 Prec 0.5073765234124439 Rec 0.98875 F1 0.6706231454005935
================== SPLIT on 0.7000000000000001
f1_

In [61]:
df_sub['pred_bi'] = (test_preds > 0.5)

print_evaluation_metrics(df_sub['humor'], df_sub['pred_bi'], '', False, 'SPLIT on '+str(split))

df_sub.to_csv('sub.csv', index=False)
df_sub.head()

================== SPLIT on 0.9
f1_score  : 0.6692047377326565
[[ 27 773]
 [  9 791]]
Acc 0.51125 Prec 0.5057544757033248 Rec 0.98875 F1 0.6692047377326565


,text,humor,pred_bi
0,New regulations mean new jobs for new regulators.,0,True
1,I have enough money for the rest of my life......,1,True
2,"Yesterday today is tomorrow, tomorrow today is...",0,True
3,waiter! One customer shouts that the ad says y...,1,True
4,I think your problem is low self-esteem. Commo...,0,True


In [62]:
print('Texts that the model failed to correctly predict:')
df_sub[df_sub['pred_bi']!=df_sub['humor']]

Texts that the model failed to correctly predict:


,text,humor,pred_bi
0,New regulations mean new jobs for new regulators.,0,True
2,"Yesterday today is tomorrow, tomorrow today is...",0,True
4,I think your problem is low self-esteem. Commo...,0,True
7,The first place to look for anything is the la...,0,True
8,"You don't really need your salary, do you? Vot...",0,True
...,...,...,...
1592,"When you come home after a long day at work, y...",0,True
1593,"""Satire is the lowest form of wit,"" as those w...",0,True
1594,"When you were born, the doctor slapped your mo...",0,True
1596,It's very satisfying to know that you've made ...,0,True
